# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [ ]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

In [ ]:
# create a CubeAPI object
cube_api = CubeAPI$new()

In [ ]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

### 1. Get Metadata Definition

In [ ]:
# retrieve metadata element
response = cube_api$get_element()

In [ ]:
str(response)

In [ ]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [ ]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

### 2. Get Bucket data from metadata collection

In [ ]:
storage_info = cube_api$get_metadata_collection_storage_info()

In [ ]:
storage_info

In [ ]:
# select a bucket
bucket_name = storage_info[21, 3]
bucket_name

In [ ]:
# file_name
file_name = storage_info[21, 4]
file_name

In [ ]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = bucket_name, file_name = file_name )
bucket_data

### 3. Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

### 4. Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### 5. Get Metadata Collection

In [ ]:
response = cube_api$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[21]][[1]]

In [ ]:
data_url = results[[21]][[1]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### 5. Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### 6. Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### 7. More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube